# 勾配法と誤差逆伝播法
ニューラルネットワークにおいて，効率よく各重みの勾配を推定することで貢献度割り当て問題を解決する方法が**誤差逆伝播法** (backpropagation) である．本節では入力層，隠れ層，出力層からなる多層ニューラルネットワークを実装し，誤差逆伝播法による勾配推定を用いて学習を行う．

コードに関しては数式と対応するような記述を行う．Deep Learningライブラリ (PyTorch, Flux.jl等) のようにLayerを定義し，それを繋げてモデルを定義するということは行わない．
Backpropagationを用いない学習法を実施することも考慮し，

In [3]:
using Base: @kwdef
#using Parameters: @unpack # or using UnPack
using LinearAlgebra, Random, Statistics, PyPlot, ProgressMeter
#using BenchmarkTools; @btime

In [4]:
struct ActivationFunction
    forward::Function   # function for forward propagation
    backward::Function  # function for back-propagation
end

(f::ActivationFunction)(x) = f.forward(x)

sigmoid = ActivationFunction(
    x -> 1.0 ./ (1.0 .+ exp.(-x)),
    y -> y .* (1 .- y)
)

tanh = ActivationFunction(
    x -> tanh.(x),
    y -> 1 .- y .^ 2
)

relu = ActivationFunction(
    x -> max.(0, x),
    y -> y .> 0
)


ActivationFunction(identity, identity)


クロスエントロピー損失関数の勾配は、ソフトマックス出力と正解ラベルの差を計算することで求められます。この過程を以下で詳しく説明します。

---

### **前提**
- **ソフトマックス出力**:  
  \( \mathbf{y} = \text{softmax}(\mathbf{z}) \)  
  各成分は以下のように定義されます:
  \[
  y_i = \frac{e^{z_i}}{\sum_{j} e^{z_j}}
  \]
  ここで \( z_i \) はモデルのロジット（未正規化スコア）です。

- **正解ラベル**:  
  \( \mathbf{t} \) は one-hot エンコーディングで表現される正解ラベルベクトル。  
  \( t_i = 1 \) （正解クラスの場合）で、それ以外は \( t_j = 0 \)（非正解クラスの場合）。

- **クロスエントロピー損失**:  
  損失関数 \( L \) は次のように定義されます:
  \[
  L = - \sum_{i} t_i \log(y_i)
  \]

---

### **勾配の導出**

クロスエントロピー損失 \( L \) を \( z_i \) で微分します。

#### **1. ソフトマックスの性質を使う**
ソフトマックスの出力 \( y_i \) に対して、ロジット \( z_k \) の微分は以下のように表されます:
\[
\frac{\partial y_i}{\partial z_k} =
\begin{cases}
y_i (1 - y_i) & \text{if } i = k, \\
-y_i y_k & \text{if } i \neq k.
\end{cases}
\]

#### **2. クロスエントロピー損失の微分**
損失 \( L \) を \( z_k \) で微分すると、チェーンルールを使って次の式が得られます:
\[
\frac{\partial L}{\partial z_k} = \sum_i \frac{\partial L}{\partial y_i} \cdot \frac{\partial y_i}{\partial z_k}.
\]

クロスエントロピー損失の \( y_i \) に関する微分は:
\[
\frac{\partial L}{\partial y_i} = -\frac{t_i}{y_i}.
\]

これを代入して整理すると:
\[
\frac{\partial L}{\partial z_k} = y_k - t_k.
\]

---

### **結果**
クロスエントロピー損失におけるロジット \( z_k \) の勾配は次のように単純化されます:
\[
\frac{\partial L}{\partial z_k} = y_k - t_k.
\]

つまり、ソフトマックス出力 \( \mathbf{y} \) と正解ラベル \( \mathbf{t} \) の差そのものです。

---

### **直感的な解釈**
- 勾配 \( y_k - t_k \) はnp.sum(np.exp(z))

# 勾配
grad = y - t
print("Softmax出力:", y)
print("勾配:", grad)
```

In [ ]:
function softmax(x; dims=2)
    expx = exp.(x .- maximum(x, dims=dims))
    return expx ./ sum(expx, dims=dims)
end

function cross_entropy_loss!(nn::NeuralNet, t::Vector)
    (;z, δ) = nn
    ŷ = softmax(z[end])
    n_batch = length(t)
    idx = CartesianIndex.([(i, t[i]) for i in 1:n_batch])
    loss = mean(log.(ŷ[idx]))

    grad = copy(ŷ)
    grad[idx] .-= 1
    δ[end] = grad / n_batch
    return loss, ŷ
end

In [43]:
y = softmax([0 1 2 5; 0 2 4 5; 0 2 4 5])

3×4 Matrix{Float64}:
 0.00626879  0.0170403  0.0463204  0.93037
 0.00473036  0.0349529  0.258269   0.702048
 0.00473036  0.0349529  0.258269   0.702048

In [38]:
t = [1, 2, 4]

3-element Vector{Int64}:
 1
 2
 4

In [14]:
abstract type NeuralNet end
(f::NeuralNet)(x) = forward(f, x)

@kwdef struct MLP <: NeuralNet
    L::Int # num. of layers
    W::Vector{Array}; b::Vector{Array};     # weights and bias
    ∇W::Vector{Array}; ∇b::Vector{Array}; # gradient of params
    z::Vector{Array}; δ::Vector{Array};   # state of forward/backward activity
    f::ActivationFunction # activation functions of hidden layers

    function MLP(num_units::Vector{Int}; activation=Sigmoid)
        L = length(num_units) - 1 # num of layers
        # initialization of parameters
        W = [2 * (rand(num_units[l], num_units[l+1]) .- 0.5) / sqrt(num_units[l]) for l in 1:L]
        b = [zeros(1, num_units[l+1]) for l in 1:L]
        
        # initialization of gradients
        ∇W, ∇b = [[zero(param[l]) for l in 1:L] for param in [W, b]]
        
        # initialization of forward / backward states
        z, δ = Vector{Array}(undef, L+1), Vector{Array}(undef, L+1)
        new(L, W, b, ∇W, ∇b, z, δ, activation)
    end;
end;

In [ ]:
"""
abstract type NeuralNet end

struct Params W::Vector{Array}; b::Vector{Array}; end # weights and bias
struct Grads W::Vector{Array}; b::Vector{Array}; end # gradient of params
struct States z::Vector{Array}; δ::Vector{Array}; end # state of forward/backward activity

@kwdef struct MLP <: NeuralNet
    L::Int # num. of layers
    params::Params
    grads::Grads
    states::States
    f::Vector{ActivationFunction} # activation functions of hidden layers

    function MLP(num_units::Vector{Int}; activations=[Sigmoid, Sigmoid])
        L = length(num_units) - 1 # num of layers
        # initialization of parameters
        params = Params(
            [2 * (rand(num_units[l], num_units[l+1]) .- 0.5) / sqrt(num_units[l]) for l in 1:L],
            [zeros(1, num_units[l+1]) for l in 1:L]
        )
        # initialization of gradients
        grads = Grads([zero(params.W[l]) for l in 1:L], [zero(params.b[l]) for l in 1:L])

        # initialization of forward / backward states
        states = States(Vector{Array}(undef, L+1), Vector{Array}(undef, L+1))
        new(L, params, grads, states, activations)
    end;
end;

# forward computation of MLP
function (mlp::MLP)(x::Array)
    (;L, f, params, states) = mlp
    (;W, b) = params
    (;z) = states
    
    z[1] = x # input (n_batch x n_neurons)
    for l in 1:L
        z[l+1] = f[l](z[l] * W[l] .+ b[l]) # hidden layers
    end
    return z[L+1] # output
end
"""

In [15]:
mlp = MLP([10, 20, 30])

MLP(2, Params([[-0.16608957301798183 0.25681758910112834 … -0.2486285917631272 -0.21077719493280464; -0.07779465491242654 0.28851295546102407 … -0.2722702267357767 -0.11226581704069914; … ; -0.2108481457354432 -0.24558923866484536 … 0.1118951052648798 0.054153677826330564; -0.0561842938073307 0.18355457870611683 … 0.018759254110035036 -0.24481841093522547], [-0.07712048869455262 -0.1591556042695856 … 0.16397868159253737 0.07163349589103427; -0.20176933726692253 -0.07311753262235113 … 0.1914520979068726 0.07351119721452141; … ; 0.1884585814975616 0.21012341290566378 … -0.05967879255515986 0.024072357631265577; 0.18055098413205684 0.03685048016179072 … 0.04043729348724201 -0.008608135881329025]], [[0.0 0.0 … 0.0 0.0], [0.0 0.0 … 0.0 0.0]]), Grads([[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]], [[0.0 0.0 … 0.0 0.0], [0.0 0.0 … 0.0 0.0]]), States(Array[#undef, #undef, #unde

In [20]:
mlp.params

Params([[-0.16608957301798183 0.25681758910112834 … -0.2486285917631272 -0.21077719493280464; -0.07779465491242654 0.28851295546102407 … -0.2722702267357767 -0.11226581704069914; … ; -0.2108481457354432 -0.24558923866484536 … 0.1118951052648798 0.054153677826330564; -0.0561842938073307 0.18355457870611683 … 0.018759254110035036 -0.24481841093522547], [-0.07712048869455262 -0.1591556042695856 … 0.16397868159253737 0.07163349589103427; -0.20176933726692253 -0.07311753262235113 … 0.1914520979068726 0.07351119721452141; … ; 0.1884585814975616 0.21012341290566378 … -0.05967879255515986 0.024072357631265577; 0.18055098413205684 0.03685048016179072 … 0.04043729348724201 -0.008608135881329025]], [[0.0 0.0 … 0.0 0.0], [0.0 0.0 … 0.0 0.0]])

mutable struct `MLP`を用意し，**重みの初期化(weight initialization)** を行う同名の関数`MLP`を用意する．重みの初期化の手法は複数あるが，ここでは重みを$W$として，$W_{ij} \sim U\left(-1/\sqrt{n}, 1/\sqrt{n}\right)$とする (Xavier initialization) \citep{Glorot2010-iu}．ただし，$n$は入力ユニット数である．

### 順伝播 (forward propagation)
$f(\cdot)$を活性化関数とする．順伝播(feedforward propagation)は以下のようになる．$(\ell=1,\ldots,L)$

$$
\begin{align}
\text{入力層 : }&\mathbf{z}_1=\mathbf{x}\\
\text{隠れ層 : }&\mathbf{a}_\ell=W_\ell \mathbf{z}_\ell +\mathbf{b}_\ell\\
&\mathbf{z}_{\ell+1}=f_\ell\left(\mathbf{a}_\ell\right)\\
\text{出力層 : }&\hat{\mathbf{y}}=\mathbf{z}_{L+1}
\end{align}
$$

In [21]:
# forward computation of MLP
function forward(mlp::MLP, x::Array)
    (;L, f, W, b, z) = mlp
    z[1] = x # input (n_batch x n_neurons)
    for l in 1:L-1
        z[l+1] = f[l](z[l] * W[l] .+ b[l]) # hidden layers
    end
    z[L+1] = z[L] * W[L] .+ b[L]
    return z[L+1] # output
end

In [22]:
x = rand(2, 10)

2×10 Matrix{Float64}:
 0.590138  0.612656  0.154764    0.083922  …  0.882684  0.160015  0.871928
 0.976392  0.554568  0.00858612  0.763462     0.959148  0.344348  0.663153

In [23]:
z = mlp(x)

2×30 Matrix{Float64}:
 0.447726  0.344038  0.549266  0.289732  …  0.575307  0.541453  0.525175
 0.45008   0.340838  0.543049  0.275432     0.586378  0.544848  0.524228

In [26]:
mlp.states.z[3]

2×30 Matrix{Float64}:
 0.447726  0.344038  0.549266  0.289732  …  0.575307  0.541453  0.525175
 0.45008   0.340838  0.543049  0.275432     0.586378  0.544848  0.524228

### 逆伝播 (backward propagation)

$i$ 番目のサンプルに対する目的関数 (objective function) あるいは損失関数 (loss function)を $\mathcal{L}^{(i)}$とする．例えば2乗誤差の場合は

$$
\mathcal{L}^{(i)}=\frac{1}{2}\left\|\hat{\mathbf{y}}^{(i)}-\mathbf{y}^{(i)}\right\|^{2}\\
$$

となる．

ニューラルネットワークの学習とは，目的関数 (objective function) あるいは損失関数 (loss function) と呼ばれる評価指標を可能な限り小さく (場合によっては大きく) するようなパラメータ集合 $\Theta = \{W_\ell, b_\ell\}_{\ell=1}^{L}$ を求める過程のことである．学習に用いられる，パラメータを探索する最適化アルゴリズムを **optimizer** という．最も単純なoptimizerである **勾配降下法** (gradient descent; GD) では全データを用いてパラメータ $\theta \in \Theta$ の更新量 $\Delta \theta$ を 

$$
\Delta \theta = -\eta \frac{\partial \mathcal{L}_{\textrm{GD}}}{\partial \theta} = -\eta \sum_{i=1}^N \frac{\partial \mathcal{L}^{(i)}}{\partial \theta}
$$

として計算する．ただし，$\mathcal{L}_{\textrm{GD}}=\sum_{i=1}^N \mathcal{L}^{(i)}$ であり，$N$は全データのサンプル数を意味する．なお，パラメータは$\theta\leftarrow \theta + \Delta \theta$により更新される．勾配降下法は$N$が大きい場合，あるいは1つのサンプルのデータサイズが大きい場合は非効率であるので，ニューラルネットワークの学習においては，データの部分集合であるミニバッチ (mini-bacth) を用いた **確率的勾配降下法** (stochastic gradient descent; SGD) が用いられる．Optimizerは複数種類があるが，一般的には $\dfrac{\partial \mathcal{L}}{\partial \theta}$ の計算を必要とする．この計算を効率よく行う手法が**誤差逆伝播法** (backpropagation) である．誤差逆伝播法は連鎖律 (chain rule; 合成関数の微分に関わる関係式) を用いて導くことができる．

$$
\begin{align}
\frac{\partial \mathcal{L}}{\partial \hat{\mathbf{y}}}&=\frac{\partial \mathcal{L}}{\partial \mathbf{z}_{L+1}}=\hat{\mathbf{y}}-\mathbf{y}\\
\delta_L&:=\frac{\partial \mathcal{L}}{\partial \mathbf{a}_L}=\frac{\partial \mathcal{L}}{\partial \mathbf{z}_{L+1}} \frac{\partial \mathbf{z}_{L+1}}{\partial \mathbf{a}_L}=\left(\hat{\mathbf{y}}-\mathbf{y}\right) \odot f_L^{\prime}\left(\mathbf{a}_L\right)\\
\mathbf{\delta}_\ell&:=\frac{\partial \mathcal{L}}{\partial \mathbf{a}_{\ell}}=\frac{\partial \mathcal{L}}{\partial \mathbf{z}_{\ell+1}} \frac{\partial \mathbf{z}_{\ell+1}}{\partial \mathbf{a}_\ell}\\
&=\left(\frac{\partial \mathcal{L}}{\partial \mathbf{a}_{\ell+1}}\frac{\partial \mathbf{a}_{\ell+1}}{\partial \mathbf{z}_{\ell+1}}\right)\frac{\partial \mathbf{z}_{\ell+1}}{\partial \mathbf{a}_{\ell}}\\
&={W_{\ell+1}}^\top \delta_{\ell+1} \odot f_\ell^{\prime}\left(\mathbf{a}_{\ell}\right)\\
\frac{\partial \mathcal{L}}{\partial W_\ell}&=\frac{\partial \mathcal{L}}{\partial \mathbf{z}_\ell} \frac{\partial \mathbf{z}_\ell}{\partial \mathbf{a}_\ell} \frac{\partial \mathbf{a}_\ell}{\partial W_\ell}=\delta_\ell \mathbf{z}_\ell^\top\\
\frac{\partial \mathcal{L}}{\partial \mathbf{b}_\ell}&=\frac{\partial \mathcal{L}}{\partial \mathbf{z}_\ell} \frac{\partial \mathbf{z}_\ell}{\partial \mathbf{a}_\ell} \frac{\partial \mathbf{a}_\ell}{\partial \mathbf{b}_\ell}=\delta_\ell
\end{align}
$$

バッチ処理を考慮すると，行列を乗ずる順番が変わる．以下では$z=f(a), g(z)=f'(a)$として膜電位を使わず，発火率情報のみを使うようにしている．このようにできない関数もあるが，今回はこのように書き下せる活性化関数のみを扱う．

$$
\begin{equation}
\frac{d}{dx} \text{Sigmoid}(x) = \text{Sigmoid}(x) \cdot \left(1 - \text{Sigmoid}(x)\right)
\end{equation}
$$
であることに注意．

$$
h\left(\mathbf{z}_{\ell+1}\right)=f^{\prime}\left(\mathbf{a}_{\ell}\right)
$$

In [ ]:
function backward!(mlp::MLP; losstype::String="binary_crossentropy")
    (;L, W, b, ∇W, ∇b, z) = mlp
    n_batch = size(z[0])[1]
    # backprop
    for l in L-1:-1:1
        δ[l] = δ[l+1] * W[l+1]' .* f.backward(z[l+1])
        ∇W[l] = z[l]' * δ[l] / n_batch
        ∇b[l] = sum(δ[l], dims=1) / n_batch
    end
end

In [ ]:
clog(x) = max(log(x), -1e2) # clamped log

function binary_crossentropy!(nn::NeuralNet, ŷ::Array, y::Array)
    (; L, states, δ) = nn
    error = ŷ - y
    loss = sum(-y .* clog.(ŷ) + (1 .- y) .* clog.(1 .- ŷ))
    δ[L] = error
    return loss
end

function squared_error!(nn::NeuralNet, ŷ::Array, y::Array)
    (;L, states, ∇f, δ) = mlp
    error = ŷ - y
    loss = sum(error .^ 2)
    δ[L] = error .* ∇f.(ŷ)
    return loss
end

### Optimizerの作成
abstract typeとして`Optimizer`タイプを作成する．

In [ ]:
abstract type Optimizer end

**確率的勾配降下法(stochastic gradient descent; SGD)** を実装する．

In [ ]:
# SGD optimizer
@kwdef struct SGD{FT} <: Optimizer
    η::FT = 1e-2
end

function optimizer_update!(param, grad, optimizer::SGD)
    @unpack η = optimizer
    param[:, :] -= η * grad
end

次に**Adam** {cite:p}`Kingma2014-fm` を実装する．

In [ ]:
# Adam optimizer
@kwdef mutable struct Adam{FT} <: Optimizer
    α::FT  = 1e-4; β1::FT = 0.9; β2::FT = 0.999; ϵ::FT = 1e-8
    ms = Dict(); vs = Dict();
end

# Adam optimizer
function optimizer_update!(param, grad, optimizer::Adam)
    @unpack α, β1, β2, ϵ, ms, vs = optimizer
    key = objectid(param)
    if !haskey(ms, key) 
        ms[key], vs[key] = zeros(size(param)), zeros(size(param))
    end    
    m, v = ms[key], vs[key]
    m += (1 - β1) * (grad - m)
    v += (1 - β2) * (grad .* grad - v)
    param[:, :] -= α * m ./ (sqrt.(v) .+ ϵ)
end

In [ ]:
function optim_step!(nn::NeuralNet, optimizer::Optimizer)
    @unpack L, W, b, ∇W, ∇b = nn
    params = [W, b]
    grads = [∇W, ∇b]
    for (param, grad) in zip(params, grads)
        for l in 1:L
            optimizer_update!(param[l], grad[l], optimizer)
        end
    end
end

In [ ]:
function train_step!(nn::NeuralNet, x::Array, y::Array, loss_fun::Function, optimizer::Optimizer=SGD())
    #ŷ = forward!(nn, x)
    ŷ = nn(x)
    loss = loss_fun(nn, ŷ, y)
    backward!(nn)
    optim_step!(nn, optimizer) # update params
    return loss
end

## Zipser-Andersenモデル
Zipser-Andersenモデル {cite:p}`Zipser1988-nc` は頭頂葉の7a野のモデルであり，網膜座標系における物体の位置と眼球位置を入力として，頭部中心座標(head centered coordinate)に変換する．隠れ層はPPC(Posterior parietal cortex)の細胞のモデルになっている．

### データセットの生成
物体位置の表現にはGaussian形式とmonotonic形式があるが，簡単のために，Gaussian形式を用いる．なお，monotonic形式については末尾の補足を参照してほしい．

In [ ]:
# Gaussian 2d
function Gaussian2d(pos, sizex=8, sizey=8, σ=1)
    x, y = 0:sizex-1, 0:sizey-1
    X, Y = [i for i in x, j in 1:length(y)], [j for i in 1:length(x), j in y]
    x0, y0 = pos
    return exp.(-((X .- x0) .^2 + (Y .- y0) .^2) / 2σ^2)
end

入力は64(網膜座標系での位置)+2(眼球位置信号)=66とする．眼球位置信号は原著ではmonotonic形式による32(=8ユニット×2(x, y方向)×2 (傾き正負))ユニットで構成されるが，簡単のために眼球位置信号も$x, y$の2次元とする．視覚刺激は-40度から40度までの範囲であり，10度で離散化する．よって，網膜座標系での位置は$8\times 8$の行列で表現される．位置は2次元のGaussianで表現する．ただし，1/e幅（ピークから1/eに減弱する幅）は15度である．$1/e$の代わりに$1/2$とすれば半値全幅(FWHM)となる．スポットサイズを$W$，Gaussianを$G(x)$とすると．$G(x+w/2)=G/e$より，$\sigma=\frac{\sqrt{2}w}{4}$と求まる．

In [ ]:
# dataset θeter
θmax = 40.0 # degree, θ∈[-θmax, θmax]
Δθ = 10.0 # degree
stimuli_size = Int(2θmax / Δθ)
w = 15.0 # degree; 1/e width
σ = √2w/(4Δθ);

# training θeter
n_data = 10000
n_traindata = Int(n_data*0.95)
n_batch = 100 # batch size
n_iter_per_epoch = Int(n_traindata/n_batch)
n_epoch = 2000; # number of epoch

In [ ]:
# generate positions
Random.seed!(0)
retinal_pos = (rand(n_data, 2) .- 0.5) * 2θmax # ∈ [-40, 40]
head_centered_pos = (rand(n_data, 2) .- 0.5) * 2θmax # ∈ [-40, 40]
#retinal_pos = (rand(n_data, 2) .- 0.5) * 2θmax # ∈ [-40, 40]
#head_centered_pos = (rand(n_data, 2) .- 0.5) * 2θmax # ∈ [-40, 40]
eye_pos = head_centered_pos - retinal_pos; # ∈ [-80, 80]

# convert
input_retina = [hcat(Gaussian2d((retinal_pos[i, :] .+ θmax)/Δθ, stimuli_size, stimuli_size, σ)...) for i in 1:n_data];
input_retina = vcat(input_retina...)
eye_pos /= 2θmax;

# concat
x_data = hcat(input_retina, eye_pos) #_encoded)
y_data = vcat([hcat(Gaussian2d((head_centered_pos[i, :] .+ θmax)/Δθ, stimuli_size, stimuli_size, σ)...) for i in 1:n_data]...);

# split
x_traindata, y_traindata = x_data[1:n_traindata, :], y_data[1:n_traindata, :]
x_testdata, y_testdata = x_data[n_traindata+1:end, :], y_data[n_traindata+1:end, :];

In [ ]:
product(sets...) = hcat([collect(x) for x in Iterators.product(sets...)]...)' # Array of Cartesian product of sets 

モデルの定義を行う．

In [ ]:
# model θeter
n_in = stimuli_size^2 + 2 # number of inputs
n_hid = 16   # number of hidden units
n_out = stimuli_size^2   # number of outputs
η = 1e-2  # learning rate
losstype = "binary_crossentropy" # "squared_error"

In [ ]:
nn = MLP([n_in, n_hid, n_out])#, bias=false)
optimizer = SGD(η=η);
loss_fun = binary_crossentropy!
#optimizer = Adam();

In [ ]:
n_in, n_hid, n_out

In [ ]:
e = 1
iter = 1
idx = (iter-1)*n_batch+1:iter*n_batch
x, y = x_traindata[idx, :], y_traindata[idx, :]

In [ ]:
size(x)

In [ ]:
out = forward!(nn, x);

In [ ]:
loss = train_step!(nn, x, y, loss_fun, optimizer)

In [ ]:
error_arr[e] += loss

学習を行う．

In [ ]:
error_arr = zeros(n_epoch); # memory array of each epoch error

@showprogress "Training..." for e in 1:n_epoch
    for iter in 1:n_iter_per_epoch
        idx = (iter-1)*n_batch+1:iter*n_batch
        x, y = x_traindata[idx, :], y_traindata[idx, :]
        loss = train_step!(nn, x, y, loss_fun, optimizer)
        error_arr[e] += loss
    end 
    error_arr[e] /= n_traindata
end

損失の変化を描画する．

In [ ]:
figure(figsize=(4,2))
#semilogy(error_arr)
plot(error_arr)
ylabel("Error"); xlabel("Epoch"); xlim(0, n_epoch)
tight_layout()

テストデータを用いて，出力を確認する．

In [ ]:
x, y = x_testdata[1:2, :], y_testdata[1:2, :]
ŷ = forward!(nn, x)

id = 1
figure(figsize=(6,2))
ax1 = subplot(1,3,1); title("input")
ax1.imshow(reshape(x[id, 1:64], (stimuli_size, stimuli_size))', interpolation="gaussian", extent=[-θmax, θmax, θmax, -θmax])
ax1.add_patch(plt.Circle((x[id, 65:66])*2θmax, radius=2, color="tab:red", fill=false))
xlabel("x"); ylabel("y");

ax2 = subplot(1,3,2); title("output")
ax2.imshow(reshape(ŷ[id, :], (stimuli_size, stimuli_size))', interpolation="gaussian", extent=[-θmax, θmax, θmax, -θmax])
ax2.add_patch(plt.Circle((x[id, 65:66])*2θmax, radius=2, color="tab:red", fill=false))
xlabel("x");

ax3 = subplot(1,3,3); title("target")
ax3.imshow(reshape(y[id, :], (stimuli_size, stimuli_size))', interpolation="gaussian", extent=[-θmax, θmax, θmax, -θmax])
ax3.add_patch(plt.Circle((x[id, 65:66])*2θmax, radius=2, color="tab:red", fill=false))
xlabel("x");

tight_layout()

重み`W1`におけるゲインフィールドの描画を行う．

In [ ]:
# Plot Gain fields
figure(figsize=(3.2, 3))
suptitle("Gain fields", fontsize=12)
subplots_adjust(hspace=0.1, wspace=0.1, top=0.925)
for i in 1:n_hid
    #subplot(3, 3, i)
    subplot(4, 4, i)
    imshow(reshape(nn.params["W"][1][1:stimuli_size^2, i], (stimuli_size, stimuli_size)), cmap="hot")
    axis("off")
end

補足としてMonotonic formatによる位置のエンコーディングに触れる．monotonic形式を入力の眼球位置と出力の頭部中心座標で用いるという仮定には，視覚刺激を中心窩で捉えた際，得られる眼球位置信号を頭部中心座標での位置の教師信号として使用できるという利点がある．\citep{Andersen1983-zp} では Parietal visual neurons (PVNs)の活動を調べ，傾き正あるいは負．0度をピークとして減少あるいは上昇の4種類あることを示した．前者は一次関数（とReLU関数）で記述可能である．

In [ ]:
get_line(p1, p2) = [(p2[2]-p1[2])/(p2[1]-p1[1]), (p2[1]*p1[2] - p1[1]*p2[2])/(p2[1]-p1[1])] # [slope, intercept]
eye_pos_coding(x; linear_θ) = relu.(linear_θ[1, :] * x .+ linear_θ[2, :])

x = -2θmax:1:2θmax
slope_θ = hcat([get_line([80, 1], [-80, -2(i-1)/stimuli_size]) for i in 1:stimuli_size]...)
y = hcat(eye_pos_coding.(x; linear_θ=slope_θ)...)
eye_pos_encoded = eye_pos_coding(-10; linear_θ=slope_θ);

In [ ]:
figure(figsize=(5,3))
subplot(2,1,1); plot(x, y'); xlabel("Eye position"); ylabel("Firing rate")
subplot(2,1,2); imshow(eye_pos_encoded[:, :]'); title(L"Eye position $=-10^\circ$"); xlabel("Units") 
tight_layout()

## 参考文献
```{bibliography}
:filter: docname in docnames
```